In [2]:
import numpy as np

np.arange(5)

array([0, 1, 2, 3, 4])

In [24]:
# encode type

n_types = 5
t = 1
board = np.array([[-1, -1, -1], [-1, 1, -1], [-1, 1, 1]])

print("board")
print(board)
print("type")
print(t)

enc_types = np.zeros(n_types, dtype=bool)
enc_types[t] = 1

enc_board = (board==1).flatten()
print("enc_board")
print(enc_board)

print("enc_types")
print(enc_types)

occ = np.hstack((enc_board, enc_types))
print("occ")
print(occ)

pow = 1<<np.arange(occ.size)
print("pow")
print(pow)

code = pow@occ
print("code")
print(code)


def encode_state(n_tiles, board, tile_type):
    enc_types = np.zeros(n_tiles, dtype=bool)
    enc_types[tile_type] = 1
    enc_board = (board==1).flatten()

    occ = np.hstack((enc_board, enc_types))
    pow = 1 << np.arange(occ.size)
    code = pow@occ

    return code


print("func output")
encode_state(n_types, board, t)



board
[[-1 -1 -1]
 [-1  1 -1]
 [-1  1  1]]
type
1
enc_board
[False False False False  True False False  True  True]
enc_types
[False  True False False False]
occ
[False False False False  True False False  True  True False  True False
 False False]
pow
[   1    2    4    8   16   32   64  128  256  512 1024 2048 4096 8192]
code
1424
func output


1424

In [31]:
n_pos = 10
n_ori = 4

tile_positions = np.arange(n_pos)
tile_orientations = np.arange(n_ori)

tile_positions = np.repeat(np.arange(n_pos, dtype=int), n_ori)
tile_orientations = np.tile(np.arange(n_ori, dtype=int), n_pos)
actions = np.vstack((tile_positions, tile_orientations)).T
actions

array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 2],
       [3, 3],
       [4, 0],
       [4, 1],
       [4, 2],
       [4, 3],
       [5, 0],
       [5, 1],
       [5, 2],
       [5, 3],
       [6, 0],
       [6, 1],
       [6, 2],
       [6, 3],
       [7, 0],
       [7, 1],
       [7, 2],
       [7, 3],
       [8, 0],
       [8, 1],
       [8, 2],
       [8, 3],
       [9, 0],
       [9, 1],
       [9, 2],
       [9, 3]])

In [18]:
import gmpy2

int(gmpy2.pack([1 for _ in range(65)], 1))

36893488147419103231